In [1]:
from tqdm import tqdm

from transport_problem import DualOracle, OptimParams
from acrcd import ACRCDOracleStacker, ACRCD_star
import numpy as np

from transport_problem import HyperParams, OptimParams
import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path
from read_data import get_network_df, get_corrs, get_full_df
from numba.core import types
import numba

from numba import njit
from graph_tool.topology import shortest_distance
import pandas as pd
import networkx as nx
import graph_tool as gt
from graph_tool.topology import shortest_distance
import numpy as np
import typing

import numba
from numba.core import types

In [2]:
# net_df = get_network_df(Path('Anahem') / 'Anaheim_net.tntp')
# corrs = get_corrs(Path('Anahem') / 'Anaheim_trips.tntp')

In [3]:
# BIG_CONST_FOR_FAKE_EDGES = 40
# print(net_df.free_flow_time.max())
# net_df.loc[net_df.free_flow_time < 1e-6, "free_flow_time"] = BIG_CONST_FOR_FAKE_EDGES
# people_count = corrs.sum()
# corrs = corrs / people_count
# net_df.capacity /= people_count
# net_df = net_df.rename({"free_flow_time": "fft"}, axis=1)
net_df = get_network_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
corrs = get_corrs(Path('SiouxFalls') / 'SiouxFalls_trips.tntp')
people_count = corrs.sum()
corrs = corrs / people_count
net_df.capacity /= people_count

In [4]:
graph = gt.Graph(net_df.values, eprops=[('capacity', 'double'), ('fft', 'double')])

l = np.maximum(np.sum(corrs, axis=1), 1e-6)
w = np.maximum(np.sum(corrs, axis=0), 1e-6)

zones_num = len(l)
sources = np.arange(zones_num)
targets = np.arange(zones_num)

params = HyperParams(gamma=10, mu_pow=0.25, rho=0.15)

In [5]:
def test_transport_problem(graph, net_df, l, w, params, sources, targets):
    dual_oracle = DualOracle(
        graph, l, w, params
    )
    oracle_stacker = ACRCDOracleStacker(
        dual_oracle,
        graph,
        sources,
        targets
    )
    
    return ACRCD_star(oracle_stacker, *oracle_stacker.get_init_vars_block(), 1000)

In [6]:
end_norm, t_calls, la_mu_calls, history, la_mu_grad_norms, t_grad_norms, x1_list, x2_list, *l_ = test_transport_problem(graph,
                                                                                                              net_df, l,
                                                                                                              w, params,
                                                                                                              sources,
                                                                                                              targets)


NameError: name 't_calls' is not defined

In [ ]:
plt.yscale('log')
plt.plot(history, label="dgap")

In [ ]:
import pandas as pd

data = []
x = t_calls
for i, j, k in zip(x, np.abs(history[1:]), np.abs(la_mu_grad_norms)):
    data.append((i, j, k))

df = pd.DataFrame(data)
df.to_csv("plot_ustm.csv", index=False)

In [ ]:
plt.yscale('log')
plt.plot(la_mu_grad_norms)

In [ ]:
x = t_calls
dual_gap = history
dual_gap_clipped = np.maximum(0, dual_gap)
                                
metric = 2 * np.array(results.history_la_mu_grad_norm) * la_mu_end_norm + dual_gap_clipped
plt.plot(x, metric, label="custom metric")
plt.yscale("log")
plt.legend()
plt.show()